<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EC%9E%A5%EA%B8%B0/%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 라이브러리 설치 및 불러오기

In [ ]:
!pip install google-cloud-vision
!pip install PyMuPDF

In [50]:
from PIL import Image
from google.cloud import vision
import numpy as np
import fitz
import cv2
import io
import os

#### 파일 저장할 폴더 생성

In [ ]:
# # 파일 저장할 폴더 생성
# os.mkdir("/content/upload_file")
# os.mkdir("/content/image_file")

#### 파일 업로드

In [73]:
# 코렙이라서 사용, flask에서는 다른 방법 필요
%cd /content/upload_file
from google.colab import files
uploaded = files.upload()

/content/upload_file


Saving 소나기002.pdf to 소나기002.pdf


In [122]:
file_names = os.listdir('/content/upload_file')
if '.ipynb_checkpoints' in file_names:
    file_names.remove('.ipynb_checkpoints')
file_names

['소나기 - 황순원.pdf']

### PDF 혹은 image 파일 ocr

##### 함수

In [116]:
def detect_paragraphs(image_path):
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/content/drive/MyDrive/api-project-397750607032-5ddc025931cd.json"
    client = vision.ImageAnnotatorClient()
    with open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    pages = response.full_text_annotation.pages
    text_result = texts[0].description

    paragraphs = []
    for page in pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                paragraph_text = ''
                bounding_box_vertices = []
                for word in paragraph.words:
                    word_text = ''.join([symbol.text for symbol in word.symbols])
                    paragraph_text += f'{word_text} '
                    bounding_box_vertices.extend([vertex for vertex in word.bounding_box.vertices])
                paragraphs.append((paragraph_text.strip(), bounding_box_vertices))
    return text_result, paragraphs, len(paragraphs)

# 문단 단락별 텍스트 가져오기
def extract_specific_paragraph(image_path):
    text_result, paragraphs, paragraphs_len = detect_paragraphs(image_path)
    paragraph_texts = []
    if paragraphs_len > 0:
        for paragraph_index in range(paragraphs_len):
            specific_paragraph_text, _ = paragraphs[paragraph_index]
            paragraph_texts.append(specific_paragraph_text)
    else:
         paragraph_texts.append("Invalid paragraph index")
    return text_result, paragraph_texts

 # PDF 파일이라면 image로 변경(코렙 아니면 경로 수정 필수)
def pdf_to_png(files):
    path = f"/content/upload_file/{files}"
    doc = fitz.open(path)
    for i, page in enumerate(doc):
        img = page.get_pixmap()
        img.save(f"/content/image_file/{i}.png")

1001 혈의 누 - 이인직
일정 전쟁(戰爭)의 총소리는 평양 일경이 떠나가는 듯하더니, 그 총소리가 그치며 사람의 자취
는 끊어지고 산과 들에 비린 티끌뿐이라.
평양성 외(外) 모란봉에 떨어지는 저녁 별은 뉘엿뉘엿 넘어가는데, 저 햇빛을 붙들어매고 싶은 마
음에 붙들어매지는 못하고 숨이 턱에 닿은 듯이 갈팡질팡하는 한 부인이 나이 삼심이 될락말락하고,
얼굴은 분을 따고 넣은 듯이 흰 얼굴이나 인정 없이 뜨겁게 내리쪼이는 가을 볕에 얼굴이 익어서 선
행듯빛이 되고, 걸음걸이는 허둥지둥하는데 옷은 흘러내려서 젖가슴이 다 드러나고 치맛자락은 땅에
점점 끌려서 걸음을 걷는 대로 치마가 밟히니 그 부인은 아무리 급한 걸음걸이를 하더라도 멀리 가
지도 못하고 허둥거리기만 한다.
남이 그 모양을 볼 지경이면 저렇게 어여쁜 젊은 여편네가 술 먹고 한길에 나와서 주정한다 할 터
이나, 그 부인은 술 먹었다 하는 말은 고사하고 미쳤다. 지랄한다 하더라도 그따위 소리는 귀에 들리
지 아니할 만하더라.
무슨 소희가 그리 대단한지 그 부인더러 불을 지경이면 대답할 여가도 없이 옥련이를 부르면서 동
아다니더라
"옥련아 우아, 옥련아 옥련아 죽었느냐 살았느냐 죽었거든 죽은 얼굴이라도 한번 다시 만나 보
자, 옥련아 옥련아, 살았거든 어미 애를 그만 쓰이고 어서 바삐 내 눈에 보이게 하여라. 옥련아, 총
에 맞아 죽었느냐, 창에 찔려 죽었느냐, 사람에게 밟혀 죽었느냐. 어리고 고운 살에 가시가 박힌 것
을 보아도 어미 된 이내 마음에 내 살이 지겹게 아프던 내 마음이라. 오늘 아침에 집에서 떠나올
때에 옥련이가 내 앞에 서서 아장아장 걸어다니면서, '어머니 어서 갑시다.' 하던 옥련이가 어디로
갔느냐."
하면서 옥련이를 찾으려고 골몰한 정신에, 옥련이보다 열 감점 스무 감점 더 소중하게 생각하는 사람
을 잃고도 모르고 옥련이만 부르며 다니다가 목이 쉬고 기운이 탈진하여 산비탈 잔디 풀 위에 털썩
주저앉았다가 혼잣말로 옥련 아버지는 옥련이 찾으려고 저 건너 산 밑으로 가더니, '어디까지 

##### 실제 데이터 사용

In [136]:
text_fully, paragraph_full_text = [], []
for file_paths in file_names:
    if file_paths[-4:] == '.pdf':
        pdf_to_png(file_paths)
        # png 파일 리스트 불러오기
        image_files = sorted(os.listdir("/content/image_file"))
        #파일 리스트에서 이미지를 불러옵니다.
        for images in image_files:
            image_path = f"/content/image_file/{images}"
            full_text, specific_paragraph_text = extract_specific_paragraph(image_path)
            text_fully.append(full_text)
            paragraph_full_text.append(specific_paragraph_text)
    else:
        file_path = f"/content/upload_file/{files}"
        full_text, specific_paragraph_text = extract_specific_paragraph(image_path)
        text_fully.append(full_text)
        paragraph_full_text.append(specific_paragraph_text)

In [142]:
# 전문 ocr 결과 리스트
print(len(text_fully), text_fully)

# 단락별 ocr 결과 리스트
print(len(paragraph_full_text),paragraph_full_text)
# 전문 중 1p 의 단락 개수
print(len(paragraph_full_text[0]))

7 ['소나기\n황순원\n소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(女)딸이라는 걸 알 수 있었다.\n소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다. 서울서는 이런 개울물을 보지\n못하기나 한 듯이.\n벌써 며칠째 소녀는 학교에서 돌아오는 길에 물장난이었다. 그런데, 어제까지 개울 기슭에\n서 하더니, 오늘은 징검다리 한가운데 앉아서 하고 있다.\n소년은 개울둑에 앉아 버렸다. 소녀가 비키기를 기다리자는 것이다.\n요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다.\n다음 날은 좀 늦게 개울가로 나왔다.\n이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다. 분홍 스웨터 소매를 걷어올\n린 목덜미가 마냥 희었다.\n한참 세수를 하고 나더니, 이번에는 물 속을 빤히 들여다 본다. 얼굴이라도 비추어 보는\n것이리라. 갑자기 물을 움켜 낸다. 고기 새끼라도 지나가는 듯.\n소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다. 그러\n나, 번번이 허탕이다. 그대로 재미있는 양, 자꾸 물만 움킨다. 어제처럼 개울을 건너는 사\n람이 있어야 길을 비킬 모양이다.\n그러다가 소녀가 물 속에서 무엇을 하나 집어 낸다. 하얀 조약돌이었다. 그리고는 벌떡 일\n어나 팔짝팔짝 징검다리를 뛰어 건너간다.\n다 건너가더니만 홱 이리로 돌아서며,\n"이 바보."\n조약돌이 날아왔다.\n소년은 저도 모르게 벌떡 일어섰다.\n단발 머리를 나풀거리며 소녀가 막 달린다. 갈밭 사잇길로 들어섰다. 뒤에는 청량한 가을\n햇살 아래 빛나는 갈꽃뿐.\n이제 저쯤 갈밭머리로 소녀가 나타나리라. 왜 오랜 시간이 지났다고 생각됐다. 그런데도\n소녀는 나타나지 않는다. 발돋움을 했다. 그러고도 상당한 시간이 지났다고 생각됐다.\n저 쪽 갈밭머리에 갈꽃이 한 옹큼 움직였다. 소녀가 갈꽃을 안고 있었다. 그리고, 이제는\n천천한 걸음이었다. 유난히 맑은 가을 햇살이 소녀의 갈꽃머리에서 반짝거렸다. 소녀 아닌\n갈꽃이 들길을 걸어가는 것만 같았다.\n소

In [143]:
# 전문 문자 개수
PDF_TEXT = ''.join(text_fully)
len(PDF_TEXT)

8115

### 요약모델